In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import re
# from sklearn.decomposition import PCA
# import joblib
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import precision_score
# from sklearn.pipeline import Pipeline
# # TF IDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score, train_test_split # fnding max_features
# # from sklearn.ensemble import
# from sklearn.decomposition import PCA
# from sklearn.metrics import classification_report
# # import joblib
# import os

# # hyperparameter tuning
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier # to look availiable hyperparameters
# from pprint import pprint
# from sklearn.model_selection import RandomizedSearchCV #

# from hyperopt import fmin, tpe, hp

# import scipy.stats as stats
# import xgboost as xgb
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV

# import spacy
# import ast
# import sklearn
# import xgboost

# import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Thesis/data/train/ner_features.csv')
test_tweet_data = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/final_tweet_data.csv')

In [ ]:
# tweet data lebeld with falcon llm and gemini llm
tweet_data_f = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/gold_tweet_test_falcon_7b.csv')
tweet_data_g = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/gold_tweet_test_gemini.csv')

In [ ]:
print(tweet_data_f['label'].value_counts()) # falcon
print(tweet_data_g['label'].value_counts()) # gemini

In [ ]:
print(tweet_data_f.shape)
print(tweet_data_g.shape)

In [ ]:
tweet_data_g.head()

In [ ]:
# modify some errors - drop columns, vaue changing
# tweet_data_f = tweet_data_f.drop('Unnamed: 0', axis = 1)
# tweet_data_g = tweet_data_g.drop('Unnamed: 0', axis = 1)

tweet_data_g['label'] = tweet_data_g['label'].apply(lambda x: re.sub('\n', ' ', x).strip())
tweet_data_g['label'] = tweet_data_g['label'].apply(lambda x: 1 if x == "Yes" else 0 )
tweet_data_g.head()

In [ ]:
# save as final dataset, which will be use for testing the model
tweet_data_f.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_falcon_WL.csv', index = False)
tweet_data_g.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_gemini_WL.csv', index = False)

In [ ]:
test_tweet_data.rename(columns={'after_removing_nl;;;' : 'Text'}, inplace = True)

In [ ]:
test_tweet_data.iloc[1].isna()

In [ ]:
def column_readjusting(df):
  i = 1
  for index, row in test_tweet_data.iterrows():
    if pd.isna(row['Text']):
      text = str(row['tweet_id'])
      clean_text = re.sub(r'^(\d+),', '', text).strip()
      df.at[index, 'Text'] = clean_text

    df.at[index, 'tweet_id'] = int(i)
    i = i + 1


In [ ]:
column_readjusting(test_tweet_data)

In [ ]:
test_tweet_data.head()

In [ ]:
test_tweet_data.to_csv('/content/drive/MyDrive/Thesis/data/test/first_tweet_datas_1.csv', index = False)

In [ ]:
new_df = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/first_tweet_datas_1.csv')

In [ ]:
new_df.tail()

In [ ]:
test_tweet_data['label'] = 0

In [ ]:
test_tweet_data.head()

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

In [ ]:
# torch.manual_seed(0)
model_name =  "tiiuae/falcon-7b-instruct"

## to reducing the memory usages:
## https://medium.com/%40rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
          model_name,
          quantization_config=quantization_config,
          #load_in_4bit=True,
          device_map = "auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# pipe = pipeline(
#     "text-generation",
#     model = model,
#     tokenizer = tokenizer,
#     torch_dtype = torch.bfloat16,
#     device_map = "auto",
#     # max_new_tokens = 20,
#     # device=0 if torch.cuda.is_available() else -1,
# )

model.eval()

In [ ]:
# 26 11:00
# Check for GPU in Colab
# import torch

# if torch.cuda.is_available():
#     print("CUDA is available. GPU is detected.")
# else:
#     print("CUDA is not available. Please ensure you are using a GPU runtime in Colab.")
#     print("Go to Runtime -> Change runtime type -> Select GPU as Hardware accelerator")

# Install multi-platform bitsandbytes for Colab
# !pip install bitsandbytes-cuda120

- create function

In [ ]:
def ad_prompt_column(row):
    # action_type = ['Physical_Action', 'Mental_Action', 'State_Change', 'Create_or_Destruction', 'Communication', 'Movement', 'Emotion_or_Feeling' , 'Perception', 'Linking(State_of_being)Verb', 'Other']
    # noise_level = ['Relevant', 'Noisy']
    # Intent = ['Statement_of_Fact', 'Opinion_or_Belief', 'Argument_or_Justification', 'Action_or_Instruction', 'No _Clear_Intent']

    # sentiment = ['Positive', 'Negative', 'Other(Unclear/Noisy)']
    text = row['Text']
    # prompt_1 = f"Is the following text '{text}' Relevant or Noise. Tell me in one word"

    # prompt_2 = f"""Classify this {text} as 'Yes' (check-worthy) or 'No' (not check-worthy) based on:
    #           - If it contains a **verifiable factual claim**.
    #           - If it is **harmful** or **misleading**"""

    # this prompt used for classify the scrapped tweet text, to add their labels
    prompt_3 = (
    "Read the following statement carefully. "
    "Your task is to determine whether the statement contains claims or information that require fact-checking. "
    "- If the statement presents factual assertions, statistics, or claims that can be verified for accuracy, respond with 'Yes'. "
    "- If the statement is purely opinion-based, general knowledge, trivial, or lacks verifiable claims, respond with 'No'.\n\n"
    f"Statement: '{text}'\nResponse:"
    )

    # prompt_for_finding_action_verb = f"Is the following text '{text}' contains any action verb, Yes or No. Tell me in one word"

    # prompt_for_catagory_of_sentence = f"Can you catagorize this following text '{text}' into one of this categories: Social, Financial, Governmental, Political, Commercial, Constitutional and Environmental"
    # f"\n1. Relevant"
    # f"\n2. Noisy"
    # f"\n Generate Only the choosen option: "

            #         Intent (Choose One):
            #         - Statement of Fact
            #         - Opinion or Belief
            #         - Argument or Justification
            #         - Action or Instruction

            #         Sentiment (Choose One):
            #         - Positive
            #         - Negative
            #         - Neutral
            #         - Other (Unclear/Noisy)

            #         Provide the response in a structured format like:
            #         Action Type: [Category]
            #         Noise Level: [Category]
            #         Intent: [Category]
            #         Sentiment: [Category]
            # """
    #response =
    #print(response)
  ############################# generating lables for tweet data ()####################################################
    inputs = tokenizer(prompt_3, return_tensors = "pt").to(model.device)
    # print(inputs)

    outputs = model.generate(**inputs, max_new_tokens = 100, do_sample = False)
    # print('+'*30)
    # print('OUTPUT')
    # print(outputs)

    generate_result = tokenizer.decode(outputs[0], skip_special_tokens = True)
    # print("generate_result")
    # print(generate_result)

    print("+"*30)

    response = re.sub('\n', ' ', generate_result).strip().split(' ')[-1]

    if response == "'Yes'":
      row['label'] = 1
    else:
      row['label'] = 0
  #####################################################################################
    # inputs = tokenizer(prompt, return_tensors = "pt").to(model.device)
    # # print(inputs)

    # outputs = model.generate(**inputs, max_new_tokens = 100, do_sample = False)
    # print('+'*30)
    # print('OUTPUT')
    # print(outputs)

    # generate_result = tokenizer.decode(outputs[0], skip_special_tokens = True)
    # print("generate_result")
    # print(generate_result)

    # print("+"*30)

    # row['Noise_text'] = False
    # row['Relevant_text'] = False

    # if 'Noise' in generate_result.split():
    #     row['Noise_text'] = True
    # elif 'Relevant' in generate_result.split():
    #     row['Relevant_text'] = True
    # return row
  ##############################################################################################
    # start_indx = generate_result.find("Action Type: ")

    # print(generate_result[start_indx:])
    return row

In [ ]:
with torch.no_grad():
  test_tweet_data = test_tweet_data.apply(ad_prompt_column, axis = 1)

In [ ]:
test_tweet_data['label'].value_counts()

In [ ]:
# save this data with lebel and without lebel
test_tweet_data.to_csv('/content/drive/MyDrive/Thesis/data/test/gold_tweet_test_falcon_7b.csv')
test_tweet_data.drop(['label'], axis = 1, inplace = True)
test_tweet_data.to_csv('/content/drive/MyDrive/Thesis/data/test/tweet_test_falcon_7b.csv')

In [ ]:
## using batch size to faster or efficient run
!nvidia-smi

In [ ]:
with torch.no_grad():
  train_df = train_df.apply(ad_prompt_column, axis = 1)

In [ ]:
train_df.to_csv('/content/drive/MyDrive/Thesis/data/train/ner_features_with_relevent_noise_prompt.csv', index = False)
# batch_size = 20
# for i in range(0, len(demo_train_df), batch_size):
#   batch_text = demo_train_df['Text'][i:i+batch_size].tolist()
#   print(batch_text)
#   results = ner_text(batch_text)
#   print(results)
#   demo_train_df.loc[i:i+batch_size-1, 'ner_result'] = results

In [ ]:
def see_the_columns_values(id, df):
    print(f"Original text: {df[df['Sentence_id'] == id]['Text'].tolist()}")
    print(f"NER: {df[df['Sentence_id'] == id]['ner_result'].tolist()}")
    # print(f"names: {df[df['Sentence_id'] == id]['names'].tolist()}")
    # print(f"organizations: {df[df['Sentence_id'] == id]['organizations'].tolist()}")
    # print(f"locations: {new_df[new_df['Sentence_id'] == id]['locations'].tolist()}")
    # print(f"dates: {new_df[new_df['Sentence_id'] == id]['dates'].tolist()}")
    # print(f"verbs: {new_df[new_df['Sentence_id'] == id]['verbs'].tolist()}")
    # print(f"lemmatized verbs: {new_df[new_df['Sentence_id'] == id]['lemmatized_verb'].tolist()}")
    # print(f"action verbs: {new_df[new_df['Sentence_id'] == id]['action_verbs'].tolist()}")
    # print(f"filtered action verbs: {new_df[new_df['Sentence_id'] == id]['filtered_action_verbs'].tolist()}")
    # print(f"joined tokens: {new_df[new_df['Sentence_id'] == id]['joined_tokens'].tolist()}")
    # print(f"count verbs: {new_df[new_df['Sentence_id'] == id]['count_verb'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['count_action_verb'].tolist()}")
    #  ###### 'names', 'organizations', 'locations','dates'
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['names'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['organizations'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['locations'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['dates'].tolist()}")

In [ ]:
see_the_columns_values(22058, demo_train_df)
# demo_train_df[demo_train_df['class_label'] == 'Yes']

In [ ]:
train_df['informative_text_res'] = train_df['Text'].apply(classify_sentence)

In [ ]:
train_df.drop(['check_ner', 'informative_text_prompt'], axis = 1, inplace=True)

In [ ]:
# train_df['informative_text_prompt'] = train_df['Text'].apply(classify_sentence)

In [ ]:
# sequences = pipe(
#     prompt,
#     max_new_tokens = 100,
#     return_full_text = False,
# )

In [ ]:
# for seq in sequences:
#   print(f"Result: {seq['generated_text']}")
x = sequences[0]['generated_text']

In [ ]:
print(x)

In [ ]:
text = "If we're $4 trillion down, we should have everything perfect, but we don't."
res = fun(text)

In [ ]:
print(res)